In [3]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
from open_spiel.python.examples.ubc_plotting_utils import parse_run, special_save_fig
from open_spiel.python.examples.ubc_evaluate_policy import run_eval

from auctions.webutils import *
from collections import Counter

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [5]:
experiment = Experiment.objects.get(name='infocompare')
runs = experiment.equilibriumsolverrun_set.all()

In [6]:
def analyze_checkpoint(checkpoint):
    record = dict()
    samples = checkpoint.evaluation.samples
    welfares = np.zeros(len(samples['rewards']['0']))
    revenues = np.zeros(len(samples['rewards']['0']))
    for player in range(checkpoint.equilibrium_solver_run.game.num_players):
        player = str(player)
        rewards = pd.Series(samples['rewards'][player])
        payments = pd.Series(samples['payments'][player])
        record[f'p{player}_utility'] = rewards.mean()
        record[f'p{player}_payment'] = payments.mean()
        welfares += rewards + payments
        revenues += payments
    record['total_welfare'] = welfares.mean()
    record['total_revenue'] = revenues.mean()

    arr = np.array(samples['allocations']['0']).astype(int) # TODO: only player 0's allocation.
    c = Counter(tuple(map(tuple, arr)))
    record['common_allocations'] = c.most_common(5)
    return record
    # print(welfares.mean(), revenues.mean())


In [10]:
records = []
for game in ['large_game_2_hide', 'large_game_2']:
    print(game)
    for run in tqdm(runs.filter(game__name=f'{game}.json')):
        checkpoint, approx_nash_conv = find_best_checkpoint(run)
        record = analyze_checkpoint(checkpoint)
        record['approx_nash_conv'] = approx_nash_conv
        record['game'] = game
        records.append(record)
    print()
df = pd.DataFrame.from_records(records)

large_game_2_hide


/tmp/ipykernel_27107/1122441073.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for run in tqdm(runs.filter(game__name=f'{game}.json')):


  0%|          | 0/10 [00:00<?, ?it/s]


large_game_2


/tmp/ipykernel_27107/1122441073.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for run in tqdm(runs.filter(game__name=f'{game}.json')):


  0%|          | 0/9 [00:00<?, ?it/s]

,p0_utility,p0_payment,p1_utility,p1_payment,total_welfare,total_revenue,common_allocations,approx_nash_conv
0,110.367535,241.088066,154.185240,252.643160,758.284001,493.731226,"[((3, 0, 0), 43006), ((3, 0, 1), 16469), ((0, ...",0.027944
1,111.235768,284.323132,134.481263,215.838837,745.879000,500.161969,"[((3, 1, 0), 51483), ((0, 3, 3), 7935), ((0, 2...",2.921965
2,85.806369,218.543831,159.341071,278.748629,742.439900,497.292460,"[((2, 1, 0), 47394), ((1, 2, 0), 9385), ((0, 2...",4.181937
3,108.768161,278.137739,131.794187,225.175112,743.875200,503.312851,"[((2, 3, 0), 23718), ((3, 1, 0), 19498), ((0, ...",3.038427
4,116.627852,291.680948,146.459485,204.897715,759.666000,496.578663,"[((3, 1, 0), 52731), ((0, 3, 3), 17339), ((3, ...",3.936240
5,93.697475,270.985124,138.074325,242.488875,745.245800,513.474000,"[((3, 1, 0), 46338), ((0, 3, 1), 12678), ((0, ...",9.109473
6,103.968094,267.771506,139.147147,235.337754,746.224500,503.109260,"[((3, 1, 0), 55282), ((0, 2, 2), 7063), ((0, 3...",7.063019
7,106.268098,253.854402,140.083727,243.348673,743.554900,497.203075,"[((2, 2, 0), 53733), ((2, 2, 1), 15249), ((0, ...",0.000000
8,109.850901,282.924599,149.076500,216.096200,757.948200,499.020799,"[((3, 1, 0), 53289), ((3, 1, 1), 19580), ((0, ...",8.871002
9,110.896876,287.534624,142.148024,216.429775,757.009300,503.964399,"[((3, 1, 0), 53622), ((0, 3, 3), 15078), ((3, ...",0.432497


In [70]:
arr = checkpoint.evaluation.samples['rewards']['0']
data = (arr,)  
from scipy.stats import bootstrap
res = bootstrap(data, np.mean, confidence_level=0.9, n_resamples=500, method='percentile')
res

BootstrapResult(confidence_interval=ConfidenceInterval(low=131.40627455988943, high=131.77069214603736), standard_error=0.11127212808392974)

In [79]:
from collections import Counter
arr = np.array(checkpoint.evaluation.samples['allocations']['0']).astype(int)
c = Counter(tuple(map(tuple, arr)))
c.most_common(len(c))
# c.most_common(5)

[((2, 3, 0), 52473),
 ((3, 1, 1), 11801),
 ((0, 3, 2), 8351),
 ((0, 3, 3), 7247),
 ((3, 1, 0), 2769),
 ((1, 3, 0), 1600),
 ((3, 0, 1), 1533),
 ((1, 3, 2), 1481),
 ((3, 0, 2), 1375),
 ((1, 3, 1), 1299),
 ((2, 2, 1), 1172),
 ((2, 2, 2), 1074),
 ((2, 1, 1), 987),
 ((2, 0, 2), 924),
 ((2, 1, 2), 793),
 ((2, 0, 3), 743),
 ((0, 3, 1), 496),
 ((1, 2, 2), 490),
 ((2, 3, 1), 475),
 ((3, 0, 0), 297),
 ((2, 0, 1), 269),
 ((2, 2, 0), 233),
 ((0, 3, 0), 232),
 ((1, 2, 1), 210),
 ((1, 1, 3), 188),
 ((1, 2, 3), 184),
 ((2, 1, 0), 136),
 ((3, 1, 2), 127),
 ((0, 2, 2), 122),
 ((1, 1, 2), 113),
 ((2, 3, 2), 89),
 ((2, 1, 3), 79),
 ((2, 0, 0), 75),
 ((3, 2, 0), 70),
 ((1, 1, 1), 67),
 ((0, 2, 3), 66),
 ((3, 2, 1), 58),
 ((1, 0, 2), 42),
 ((1, 2, 0), 31),
 ((3, 3, 0), 31),
 ((1, 0, 3), 26),
 ((2, 2, 3), 25),
 ((3, 0, 3), 23),
 ((0, 2, 0), 17),
 ((3, 2, 2), 15),
 ((0, 1, 3), 15),
 ((3, 3, 1), 13),
 ((0, 2, 1), 12),
 ((0, 1, 2), 12),
 ((1, 0, 1), 10),
 ((0, 1, 1), 8),
 ((1, 1, 0), 8),
 ((1, 3, 3), 4),
 ((0,

In [73]:
# data

In [87]:
x = df.copy()
del x['common_allocations']
x

,p0_utility,p0_payment,p1_utility,p1_payment,total_welfare,total_revenue,approx_nash_conv
0,119.299547,260.433953,151.828569,223.894231,755.456300,484.328184,6.136778
1,110.283074,283.580826,142.192599,218.300201,754.356700,501.881027,4.596013
2,108.893819,280.877781,147.757828,219.723572,757.253000,500.601353,0.565621
3,88.345236,204.572365,151.576142,291.493358,735.987100,496.065723,10.670377
4,128.823059,303.109242,136.434884,189.175516,757.542700,492.284757,1.434662
5,118.516735,291.097365,144.563827,205.458673,759.636599,496.556037,1.962268
6,100.129506,278.194794,147.068753,229.939047,755.332100,508.133841,9.096962
7,112.897518,301.943282,131.527484,205.951316,752.319600,507.894599,3.092972
8,131.585773,293.050027,131.015494,190.298906,745.950200,483.348932,3.164699


In [86]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(df['common_allocations'])

0     [((3, 1, 0), 48502), ((3, 0, 1), 15740), ((0, 2, 3), 9116), ((2, 1, 1), 5702), ((2, 2, 0), 2920)]
1     [((3, 1, 0), 51059), ((3, 1, 1), 18585), ((0, 2, 3), 2915), ((1, 0, 3), 2265), ((1, 2, 0), 2030)]
2     [((3, 1, 0), 52859), ((3, 0, 2), 10133), ((3, 1, 1), 7589), ((0, 3, 3), 6406), ((0, 3, 0), 2300)]
3     [((2, 1, 0), 53382), ((0, 2, 2), 19880), ((0, 3, 1), 5349), ((2, 0, 1), 2844), ((0, 3, 2), 2425)]
4      [((3, 2, 0), 67191), ((0, 1, 3), 9668), ((0, 3, 3), 3597), ((0, 3, 2), 2545), ((3, 1, 0), 1947)]
5    [((3, 1, 0), 52412), ((0, 3, 3), 16876), ((3, 0, 2), 16313), ((3, 1, 1), 5705), ((3, 2, 0), 1191)]
6      [((3, 1, 0), 57387), ((3, 1, 1), 5073), ((3, 0, 1), 4793), ((2, 1, 1), 4097), ((0, 1, 3), 3189)]
7      [((3, 1, 0), 52099), ((0, 3, 3), 8713), ((3, 1, 1), 4749), ((3, 2, 0), 3904), ((1, 3, 3), 2717)]
8     [((2, 3, 0), 52473), ((3, 1, 1), 11801), ((0, 3, 2), 8351), ((0, 3, 3), 7247), ((3, 1, 0), 2769)]
Name: common_allocations, dtype: object

In [75]:
df['common_allocations']

0    [((3, 1, 0), 48502), ((3, 0, 1), 15740), ((0, ...
1    [((3, 1, 0), 51059), ((3, 1, 1), 18585), ((0, ...
2    [((3, 1, 0), 52859), ((3, 0, 2), 10133), ((3, ...
3    [((2, 1, 0), 53382), ((0, 2, 2), 19880), ((0, ...
4    [((3, 2, 0), 67191), ((0, 1, 3), 9668), ((0, 3...
5    [((3, 1, 0), 52412), ((0, 3, 3), 16876), ((3, ...
6    [((3, 1, 0), 57387), ((3, 1, 1), 5073), ((3, 0...
7    [((3, 1, 0), 52099), ((0, 3, 3), 8713), ((3, 1...
8    [((2, 3, 0), 52473), ((3, 1, 1), 11801), ((0, ...
Name: common_allocations, dtype: object

In [82]:
df['common_allocations'].loc[0]

[((3, 1, 0), 48502),
 ((3, 0, 1), 15740),
 ((0, 2, 3), 9116),
 ((2, 1, 1), 5702),
 ((2, 2, 0), 2920)]

In [83]:
df['common_allocations'].loc[1]

[((3, 1, 0), 51059),
 ((3, 1, 1), 18585),
 ((0, 2, 3), 2915),
 ((1, 0, 3), 2265),
 ((1, 2, 0), 2030)]

In [78]:
df['common_allocations'].loc[6]

[((3, 1, 0), 57387),
 ((3, 1, 1), 5073),
 ((3, 0, 1), 4793),
 ((2, 1, 1), 4097),
 ((0, 1, 3), 3189)]

In [117]:
from open_spiel.python.examples.ubc_utils import *

g_db = Game.objects.get(name='tiny.json')
g = load_game(g_db)
it = g.new_initial_state().child(0).child(0).child(1).child(1).information_state_tensor()

get_player_type_index(g_db.num_players, g_db.num_actions, g_db.num_products)
get_player_type(g_db.num_players, g_db.num_actions, g_db.num_products, it)

Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.


[110.0, 150.0]

CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/tiny.json
Done config parsing


In [100]:
it[145:155]

[0.0, 500.0, 120.0, 80.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [108]:
it[146]

500.0

In [115]:
Game.objects.all()

<QuerySet [<Game: tiny.json>, <Game: large_game_2.json>, <Game: large_game_2_hide.json>]>

In [10]:

Experiment.objects.all()

<QuerySet [<Experiment: minitest>, <Experiment: infocompare>]>

In [9]:
from collections import defaultdict
eq_checkpoints = defaultdict(dict) # Player -> Seed -> Agent
for run in runs.filter(game__name=f'large_game_2.json'):
    checkpoint, approx_nash_conv = find_best_checkpoint(run)
    print(checkpoint)
    for player in range(2):
        env_and_model = db_checkpoint_loader(checkpoint)
        eq_checkpoints[player][checkpoint.equilibrium_solver_run.config['seed']] = dict(agent=env_and_model.agents[player], reward=checkpoint.evaluation.mean_rewards[player], checkpoint=checkpoint)
                                                                                        


large_game_2-feb22mlp-101 Iteration 4000000


Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing


large_game_2-feb22mlp-105 Iteration 7000000


Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing


large_game_2-feb22mlp-106 Iteration 9000000


Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing


large_game_2-feb22mlp-100 Iteration 9000000



KeyboardInterrupt



In [71]:
# For each player
#     For each of the equilibria
#         For each other equilibrium strategy + straightforward bidding
#             performance_delta = performance - old_performance
N_SAMPLES = 10_000

records = [] 
for player in eq_checkpoints.keys():
    for seed in tqdm(eq_checkpoints[player].keys()):
        for other_seed in eq_checkpoints[player].keys():
            # TODO: Add straightforward
            data = eq_checkpoints[player][seed]
            performance_old = data['reward']
            env_and_model = db_checkpoint_loader(data['checkpoint'])
            env_and_model.agents[player] = eq_checkpoints[player][other_seed]['agent'] 
            eval_output = run_eval(env_and_model, N_SAMPLES)
            performance_new = pd.Series(eval_output['rewards'][player]).mean()
            performance_delta = performance_new - performance_old 

            record = {
                'player': player,
                'seed': seed,
                'vs_seed': other_seed,
                'old_reward': performance_old,
                'new_reward': performance_new,
                'performance_delta': performance_delta,
            }

            records.append(record)
                
df = pd.DataFrame.from_records(records)

/tmp/ipykernel_27107/1162670245.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for seed in tqdm(eq_checkpoints[player].keys()):


  0%|          | 0/9 [00:00<?, ?it/s]

Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set

  0%|          | 0/9 [00:00<?, ?it/s]

Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set

In [8]:
runs

<QuerySet [<EquilibriumSolverRun: large_game_2-feb22mlp-101>, <EquilibriumSolverRun: large_game_2-feb22mlp-105>, <EquilibriumSolverRun: large_game_2-feb22mlp-106>, <EquilibriumSolverRun: large_game_2-feb22mlp-100>, <EquilibriumSolverRun: large_game_2-feb22mlp-102>, <EquilibriumSolverRun: large_game_2-feb22mlp-109>, <EquilibriumSolverRun: large_game_2-feb22mlp-108>, <EquilibriumSolverRun: large_game_2-feb22mlp-103>, <EquilibriumSolverRun: large_game_2-feb22mlp-107>, <EquilibriumSolverRun: large_game_2_hide-feb22mlp-100>, <EquilibriumSolverRun: large_game_2_hide-feb22mlp-103>, <EquilibriumSolverRun: large_game_2_hide-feb22mlp-101>, <EquilibriumSolverRun: large_game_2_hide-feb22mlp-107>, <EquilibriumSolverRun: large_game_2_hide-feb22mlp-109>, <EquilibriumSolverRun: large_game_2_hide-feb22mlp-104>, <EquilibriumSolverRun: large_game_2_hide-feb22mlp-108>, <EquilibriumSolverRun: large_game_2_hide-feb22mlp-105>, <EquilibriumSolverRun: large_game_2_hide-feb22mlp-106>, <EquilibriumSolverRun: lar

In [72]:
pd.Series(player_to_deltas[0]).describe()

/tmp/ipykernel_27107/3666543712.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(player_to_deltas[0]).describe()


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
dtype: float64

In [73]:
pd.Series(player_to_deltas[1]).describe()

/tmp/ipykernel_27107/1540592524.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(player_to_deltas[1]).describe()


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
dtype: float64

In [74]:
for seed in tqdm(eq_checkpoints[player].keys()):
    print(seed)

/tmp/ipykernel_27107/2763725693.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for seed in tqdm(eq_checkpoints[player].keys()):


  0%|          | 0/9 [00:00<?, ?it/s]

101
105
106
100
102
109
108
103
107


In [75]:
df.groupby('player')['performance_delta'].describe()

,count,mean,std,min,25%,50%,75%,max
player,,,,,,,,
0,81.0,-11.977054,9.671311,-47.770907,-16.994656,-10.881859,-5.392025,2.215120
1,81.0,-8.654883,7.770229,-30.379849,-12.397718,-8.364801,-2.154180,4.478757


In [76]:
df.query('seed == vs_seed').groupby('player')['performance_delta'].describe()

,count,mean,std,min,25%,50%,75%,max
player,,,,,,,,
0,9.0,-0.235891,0.552249,-1.098782,-0.682681,-0.024914,0.047093,0.463176
1,9.0,-0.034388,0.310164,-0.406187,-0.332099,-0.075447,0.223776,0.414846


In [77]:
df.query('seed != vs_seed').groupby(['player', 'seed'])['performance_delta'].describe()

count       mean        std        min        25%        50%  \
player seed                                                                 
0      100     8.0 -10.976534   3.444491 -14.707008 -13.748401 -11.741662   
       101     8.0 -11.077191   5.636414 -23.097940 -12.315339  -9.337681   
       102     8.0 -29.829560  10.227389 -47.770907 -36.928552 -25.920123   
       103     8.0  -7.596450   8.181102 -20.257594 -13.873464  -6.110278   
       105     8.0 -12.175285   7.391471 -23.923297 -16.942546  -9.590041   
       106     8.0 -14.160652   7.392769 -24.212069 -18.915829 -14.057188   
       107     8.0 -14.446833   4.774743 -24.368007 -15.851214 -13.717332   
       108     8.0  -6.435267   7.279367 -19.191882  -9.918846  -5.753248   
       109     8.0 -14.304521   7.009755 -25.629653 -19.267594 -12.963406   
1      100     8.0  -5.372832   6.245821 -14.252609  -9.575402  -5.774661   
       101     8.0 -14.293091   6.719030 -30.379849 -13.582201 -12.515852   
       102     8.0  -7.308610   7.018489 -22.729784  -8.351728  -6.636873   
       103     8.0  -7.196544   6.308559 -14.408702 -10.971276  -8.762360   
       105     8.0 -12.831459   7.185415 -28.488613 -14.871679 -11.371393   
       106     8.0  -8.519752   6.559038 -21.891695 -10.689255  -8.091657   
       107     8.0 -14.647678   7.470911 -24.612063 -20.776071 -13.226450   
       108     8.0  -4.578035   8.139116 -23.526681  -4.777826  -2.312325   
       109     8.0 -12.844001   7.158428 -27.003535 -15.271181 -10.981216   

                   75%        max  
player seed                        
0      100   -8.380607  -5.217149  
       101   -7.234566  -5.960795  
       102  -22.299686 -18.651266  
       103   -0.659864   2.215120  
       105   -6.227127  -5.227164  
       106   -8.504858  -5.216866  
       107  -11.216521  -9.097336  
       108   -1.383315   1.712915  
       109  -10.511245  -3.496376  
1      100   -1.051572   4.478757  
       101  -11.246030  -8.935073  
       102   -3.052795   0.270212  
       103   -5.022234   3.660667  
       105   -7.927565  -6.291300  
       106   -3.835902  -1.081784  
       107  -10.195578  -3.699110  
       108   -0.245096   1.858627  
       109   -8.410454  -4.736632

In [82]:
df

,player,seed,vs_seed,old_reward,new_reward,performance_delta
0,0,101,101,119.299547,119.746834,0.447287
1,0,101,105,119.299547,109.610856,-9.688692
2,0,101,106,119.299547,113.338752,-5.960795
3,0,101,100,119.299547,96.201607,-23.097940
4,0,101,102,119.299547,112.053456,-7.246092
...,...,...,...,...,...,...
157,1,107,102,131.015494,107.529890,-23.485604
158,1,107,109,131.015494,127.316384,-3.699110
159,1,107,108,131.015494,119.542161,-11.473333
160,1,107,103,131.015494,116.035928,-14.979566


In [1]:
for player in range(2):
    utility_differences = df.query(f'player == {player}').sort_values(['seed', 'vs_seed']).performance_delta.values.reshape((9, 9))
    divnorm=plt.matplotlib.colors.TwoSlopeNorm(vmin=-25., vcenter=0., vmax=25)
    plt.imshow(utility_differences, norm=divnorm, cmap='RdBu')
    cbar = plt.colorbar(extend='min')
    plt.xticks(range(9))
    plt.yticks(range(9))
    plt.xlabel('playing strategy from equilibrium')
    plt.ylabel('against equilibrium')
    plt.title(f'Player {player}')
    cbar.set_label('Change in utility')
    plt.show()
    special_save_fig(plt.gcf(), f'player_{player}.pdf')

NameError: name 'df' is not defined

In [12]:
EquilibriumSolverRunCheckpoint.objects.filter(equilibrium_solver_run=12)

<QuerySet []>

In [4]:
game = Game.objects.last()

In [10]:
Experiment.objects.filter(equilibriumsolverrun__game=game).distinct()

<QuerySet [<Experiment: infocompare>]>

In [12]:
EquilibriumSolverRun.objects.last().pk

32

In [13]:
game = Game.objects.last()